In [1]:
import numpy as np
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt
PointConfiguration.set_engine('internal')
from numpy import linalg as LA

# 1-1. Calculating Hilbert Series and Calabi-Yau Volume

In [2]:
def dictionary(x):
    dict1, dict2 = {},{}
    for i in range(len(x)):
        dict1[i] = x[i]
        dict2[i] = [1, x[i][0],x[i][1]]
    return dict1, dict2

In [3]:
def clck_chck(x):
    a, b, c = x[0], x[1], x[2]
    det = b[0]*c[1]-b[1]*c[0]-a[0]*c[1]+a[1]*c[0]+a[0]*b[1]-a[1]*b[0]
    if det < 0:
        return 1
    return 0

In [4]:
def triangle(x):
    if [0,0] not in x:
        x.append([0,0])
    dict1, dict2 = dictionary(x)
    p = PointConfiguration(x)
    t = np.array(p.triangulate())
    for i in range(len(t)):
        if clck_chck([dict1[t[i][0]],dict1[t[i][1]],dict1[t[i][2]]]):
            t[i][0], t[i][2] = t[i][2], t[i][0]
    tri, vec = [], []
    for i in range(len(t)):
            tri.append([dict1[t[i][0]],dict1[t[i][1]],dict1[t[i][2]]])
            vec.append([dict2[t[i][0]],dict2[t[i][1]],dict2[t[i][2]]])
    return tri, vec

In [5]:
def normal(x):
    cross_prod = []
    for i in range(len(x)):
        cross_prod.append([np.cross(x[i][-1],x[i][0]),np.cross(x[i][0],x[i][1]),np.cross(x[i][1],x[i][2])])
    return cross_prod

In [6]:
def Hilbert_series(x):
    var('p1','p2','p3')
    tri, vec = triangle(x)[0], triangle(x)[1]
    nor = normal(vec)
    f(p1,p2,p3) = 0
    for i in range(len(nor)):
        den(p1,p2,p3) = 1
        for j in range(3):
            den(p1,p2,p3) = den(p1,p2,p3) * (1-p1^nor[i][j][0]*p2^nor[i][j][1]*p3^nor[i][j][2])
        f(p1,p2,p3) =  f(p1,p2,p3) + 1/den(p1,p2,p3)
    return f(p1,p2,p3)
        

In [7]:
def Calabi_Yau_Volume(x):
    var('b1','b2','b3','t')
    series = Hilbert_series(x)
    series_ = series.subs(p1 = exp(-b1*t), p2 = exp(-b2*t), p3 = exp(-b3*t))
    C = t^3*series_
    V = limit(C,t=0)
    return V


## 1-2 Experiment

### 1-2-1. The Conifold

In [8]:
Conifold_Vol = Calabi_Yau_Volume([[0,0],[0,1],[1,1],[1,0]])

In [9]:
bool(Conifold_Vol == b1/b2/b3/(b1-b2)/(b1-b3))

True

### 1-2-2. The first del Pezzo surface

In [10]:
first_del_Pezzo_surface_Vol = Calabi_Yau_Volume([[-1,-1],[-1,0],[0,1],[1,0]])

In [11]:
bool(first_del_Pezzo_surface_Vol == 2*(4*b1+2*b2-b3)/(b1+b2)/(b1-b2+2*b3)/(b1-b2-b3)/(b1+b2-b3))

True

### 1-2-3. The second del Pezzo surface

In [12]:
calabi_2nd_del_num = 7*b1^2 + 2*b1*b2+2*b1*b3-b2^2-b3^2+2*b2*b3
calabi_2nd_del_den = (b1+b2)*(b1+b2-b3)*(b1-b2-b3)*(b1-b2+b3)*(b1+b3)
calabi_2nd_del(b1,b2,b3) = calabi_2nd_del_num/calabi_2nd_del_den

In [13]:
second_del_Pezzo_surface_Vol = Calabi_Yau_Volume([[-1,-1],[-1,0],[0,1],[1,0],[0,-1]])

In [14]:
bool(second_del_Pezzo_surface_Vol == calabi_2nd_del)

True

# 2-1. Minimization Process

## 2-2. Experiment

### 2-2-1. The Conifold

In [15]:
Conifold_Vol_ = Conifold_Vol(b1=3)

In [16]:
Conifold_Vol_

1/((b2 - 3)*b3^2 - 3*b2^2 + (b2^2 - 9*b2 + 18)*b3 + 18*b2 - 27) - 1/(b2*b3^2 + (b2^2 - 3*b2)*b3)

### 2-2-2. The first del Pezzo surface

In [17]:
first_del_Pezzo_surface_Vol_ = first_del_Pezzo_surface_Vol(b1=3)

In [18]:
plot3d(first_del_Pezzo_surface_Vol_,(b2,-10,10),(b3,-10,10))

Graphics3d Object

### 2-2-3. The second del Pezzo surface

In [19]:
second_del_Pezzo_surface_Vol

-1/(b1^3 - 2*b1^2*b2 + b1*b2^2 + (5*b1 - 3*b2)*b3^2 + 2*b3^3 + (4*b1^2 - 5*b1*b2 + b2^2)*b3) - 1/((b1 + b2)*b3^2 - (b1*b2 + b2^2)*b3) + 1/((b1 - 3*b2)*b3^2 + 2*b3^3 - (b1*b2 - b2^2)*b3) + 1/(b2*b3^2 - (b1*b2 + b2^2)*b3) - 1/(b2*b3^2 - (b1*b2 - b2^2)*b3)

In [20]:
plot3d(second_del_Pezzo_surface_Vol(b1=3),(b2,-10,10),(b3,-10,10))

Graphics3d Object

In [21]:
def min_volume(func):
    func = func(b1=3)
    diff_ = np.sqrt(diff(func,b2)**2 + diff(func,b3) **2)
    def obj(v):
        x,y = v
        try:
            return diff_(b2=x,b3=y)
        except ValueError:
            return 100
    def obj_(v):
        x,y = v
        try:
            return func(b2=x,b3=y)
        except ValueError:
            return 100


    bounds = [[i, i+1] for i in range(-3,3)]
    b1b2 = []
    num = 0
    for i in bounds:
        num += 1
        for j in bounds:
            result = differential_evolution(obj, [i,j])
            b1b2.append(result['x'])
        print(float(100*num/len(bounds)),'%')
    min_cand = []

    for v in b1b2:
        
        if obj((v[0],v[1])) < 1e-5:
            min_cand.append(obj_((v[0],v[1])))
    min_index = b1b2[np.argmin(min_cand)]
    print('minimum value :', np.min(min_cand))
    print('minimum index([b2,b3]):',min_index )



In [22]:
min_volume(second_del_Pezzo_surface_Vol)

16.666666666666668 %
33.333333333333336 %
50.0 %
66.66666666666667 %
83.33333333333333 %
100.0 %
minimum value : 0.2514201464984889
minimum index([b2,b3]): [-2.00010141 -2.00049312]
